## Samples Embedding with CamemBERT then samples classification by FLAIR TextClassifier

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import flair
from flair.data import Sentence
from flair.embeddings import CamembertEmbeddings
from flair.embeddings import TransformerWordEmbeddings
import pathlib
import os
from flair.data import Corpus
from flair.datasets import CSVClassificationCorpus

In [2]:
import torch
device = None
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cuda')

In [3]:
# Charger les jeux de données : test = jeu d'évaluation final pour flair, 
# Annoté sans active learning, parce que l'AL pourrait entraîner un biais.

train_set = pd.read_csv('train.csv', sep=';', encoding='utf-8')
dev_set = pd.read_csv('dev.csv', sep=';', encoding='utf-8')
test_set = pd.read_csv('test.csv', sep=';', encoding='utf-8')

train_set = train_set[['index', 'exemple', 'label']].set_index('index')
test_set = test_set[['index', 'exemple', 'label']].set_index('index')
dev_set = dev_set[['index', 'exemple', 'label']].set_index('index')

# Embedding

In [ ]:
# Camembert Embedding

embedding = CamembertEmbeddings()

## Load datasets

In [ ]:
# Corpus is a tool from Flair to load train and test set.

# 1. get the corpus
path = os.getcwd()

# this is the folder in which train, test and dev files reside
data_folder = path = os.getcwd()

# column format indicating which columns hold the text and label(s)
column_name_map = {2: "text", 3: "label_topic"}

# load corpus containing training, test and dev data and if CSV has a header, you can skip it
corpus: Corpus = CSVClassificationCorpus(data_folder,                                        
                                         column_name_map,
                                         skip_header=True,
                                         delimiter=';') 

# look at what the datasets look like
import flair.datasets 
stats = corpus.obtain_statistics()
print(stats)

In [6]:
# 2. create the label dictionary
label_dict = corpus.make_label_dictionary()

2021-08-12 11:36:42,610 Computing label dictionary. Progress:


100%|██████████| 2830/2830 [00:00<00:00, 4365.76it/s]

2021-08-12 11:36:43,367 [b'0', b'1']


In [7]:
# 3. make a list of word embeddings
word_embeddings = [embedding]

In [8]:
from flair.embeddings import DocumentRNNEmbeddings

# 4. initialize document embedding by passing list of word embeddings
# Can choose between many RNN types (GRU by default, to change use rnn_type parameter)
document_embeddings: DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embeddings,
                                                                   hidden_size=512,
                                                                   reproject_words=True,
                                                                   reproject_words_dimension=256,
                                                                   )

# TextClassifier

In [9]:
from flair.models import TextClassifier

In [10]:
# 5. create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

In [11]:
from flair.trainers import ModelTrainer

In [12]:
# 6. initialize the text classifier trainer
trainer = ModelTrainer(classifier, corpus)

In [ ]:
# 7. start the training
trainer.train('resources/taggers/complication_kt',
              learning_rate=0.2
              mini_batch_size=16,
              anneal_factor=0.5,
              patience=5,
              max_epochs=112,
              embeddings_storage_mode='gpu')

In [ ]:
# Plot training curves
from flair.visual.training_curves import Plotter
plotter = Plotter()

plotter.plot_training_curves('./resources/taggers/complication_kt/loss.tsv')

plotter.plot_weights('./resources/taggers/complication_kt/weights.txt')



In [ ]:
# Once the model is trained you can load it to predict the class of new sentences. 

# Just call the predict method of the model.

classifier = TextClassifier.load('resources/taggers/complication_kt/final-model.pt')


# Create example sentence

sentence = Sentence('complication precoce : ACR en salle de catheterimse reanime par adrenaline et massage cardiaque no flow 1 min low flow 5 min')

# Predict class and print

classifier.predict(sentence)

print(sentence.labels)

# Evaluation

## On test set

In [16]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sn
import math

In [17]:
# Predict samples label

y_pred = []
for row in test_set.iterrows():
    #print("Train example:", row[1].exemple)
    sentence = Sentence(row[1].exemple)
    classifier.predict(sentence)
    #print("Predicted class", sentence.labels)
    y_pred.append(sentence.labels)
    
y_pred = pd.DataFrame(y_pred)
df = y_pred.columns=['result']

In [18]:
df = y_pred['result'].astype(str).str[:1].astype(int)

y_pred = df.tolist()

In [19]:
y_true = test_set["label"]

y_true = y_true.tolist()

In [ ]:
# Metrics

F1 = f1_score(y_true, y_pred, average='micro')
F2 = f1_score(y_true, y_pred, average=None)
RC1 = recall_score(y_true, y_pred, average='micro')
RC2 = recall_score(y_true, y_pred, average=None)
PC1 = precision_score(y_true, y_pred, average='micro')
PC2 = precision_score(y_true, y_pred, average=None)
AC = accuracy_score(y_true, y_pred)

print('global F1-score')
print(F1)
print('F1-score by class')
print(F2)
print('Recall global')
print(RC1)
print('Recall by class')
print(RC2)
print('Precision global')
print(PC1)
print('Precision by class')
print(PC2)
print('Accuracy')
print(AC)

from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_true, y_pred)
print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

In [ ]:
# Matrice

CM = confusion_matrix(y_true, y_pred, labels=[1, 0])

array = CM

labels=[1, 0]

print(CM)

fig = plt.gcf()
df_cm = pd.DataFrame(array, index=labels, columns=labels)
sn.set(font_scale=1.4)
sn.heatmap(df_cm,  annot=True, annot_kws={"size": 16}, cmap="YlGnBu", fmt='g') # font size
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.show()
fig.savefig('CM_flair.pdf', bbox_inches='tight')

In [22]:
# A table to look at results on test samples
test = test_set.reset_index()
resultats_test = test.join(df)
resultats_test.to_csv("resultats_test.csv", sep=';', encoding='utf-8')

In [23]:
# To get patient wise result (on patients that are on test set)

In [ ]:
# To link correspondence table and results on test set

# Cf table
cf = pd.read_csv('table_de_correspondance.csv', 
                 sep=';', 
                 encoding='utf-8',
                index_col=0)

print(cf.IPP.nunique())

# Result table
res = pd.read_csv('resultats_test.csv', 
                 sep=';', 
                 encoding='utf-8')
res['index'] = res['index'].str.extract('(\d+)')
res = res[['index', 'exemple', 'label', 'result']]
res['index'] = res['index'].astype(int)
res = res.set_index('index')

# Left merge on result to get patient ID
df = pd.merge(res, cf, how='left', left_index=True, right_index=True)

print(df)
print(df.IPP.nunique())

In [ ]:
# Link every sample to its patient
df.groupby(['IPP', 'label']).groups

# We consider that if a patient had at least one sample positive he should be count as a complicated procedure 
compare = df.groupby(['IPP'])["result", "label"].apply(lambda x : x.astype(int).sum())
compare['label'][compare['label'] > 0] = 1
compare['result'][compare['result'] > 0] = 1
compare['erreur'] = compare['label'] + compare['result']

# To have a look on compare table
compare.to_csv('compare_par_IPP.csv', sep=";", encoding='utf-8')

# erreur = 0 -> True negative, error = 2 -> True positive, error = 1 soit False negative or False positive.
print(compare)

In [ ]:
# Metrics

yp_true = compare["label"]
yp_true = yp_true.tolist()

yp_pred = compare["result"]
yp_pred = yp_pred.tolist()

F1 = f1_score(yp_true, yp_pred, average='micro')
F2 = f1_score(yp_true, yp_pred, average=None)
RC1 = recall_score(yp_true, yp_pred, average='micro')
RC2 = recall_score(yp_true, yp_pred, average=None)
PC1 = precision_score(yp_true, yp_pred, average='micro')
PC2 = precision_score(yp_true, yp_pred, average=None)
AC = accuracy_score(yp_true, yp_pred)

print('global F1-score')
print(F1)
print('F1-score par classe')
print(F2)
print('Recall global')
print(RC1)
print('Recall par classe')
print(RC2)
print('Precision global')
print(PC1)
print('Precision par classe')
print(PC2)
print('Accuracy')
print(AC)

from sklearn.metrics import average_precision_score
average_precision = average_precision_score(yp_true, yp_pred)
print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

In [ ]:
# Matrice

CM = confusion_matrix(yp_true, yp_pred, labels=[1, 0])

array = CM

labels=[1, 0]

print(CM)

fig = plt.gcf()
df_cm = pd.DataFrame(array, index=labels, columns=labels)
sn.set(font_scale=1.4)
sn.heatmap(df_cm,  annot=True, annot_kws={"size": 16}, cmap="Oranges", fmt='g') # font size
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.show()
fig.savefig('CM_patient_flair.pdf', bbox_inches='tight')

# CLASSIFICATION of UNLABELED SAMPLES

When Text classifier results are adequate. You can load unclassified samples and let the model gives them a label.

In [ ]:
# Load unlabeled samples

unlab_set = pd.read_csv('unlab.csv', sep =';', encoding = "utf-8").set_index('Unnamed: 0')

In [ ]:
# Classify unlabeled

unlab = []
for row in unlab_set.iterrows():
    #print("Train example:", row[1].exemple)
    sentence = Sentence(row[1].exemple)
    classifier.predict(sentence)
    #print("Predicted class", sentence.labels)
    unlab.append(sentence.labels)
    
unlab = pd.DataFrame(unlab)
unlab = unlab_set.join(unlab)

In [ ]:
# Saved classified samples in a csv document that will be used for the score estimation

unlab.to_csv("unlab_classifier.csv", sep=';', encoding = 'utf-8')